In [1]:
import os
def get_packages_folders():
    appdata = [('Steam', '{APPDATA}\\Microsoft Flight Simulator\\usercfg.opt'), ('Store', '{LOCALAPPDATA}\\Packages\\Microsoft.FlightSimulator_8wekyb3d8bbwe\\LocalCache\\usercfg.opt')]
    packages = []
    for source in appdata:
        file_name = source[1].format(**os.environ)
        if os.path.isfile(file_name):
            for line in open(file_name):
                if line.startswith('InstalledPackagesPath'):
                    packages.append((source[0], line.split('"')[1]))
    print('Found:', packages)
    return packages
packages = get_packages_folders()

Found: [('Steam', 'd:\\MSFS\\Microsoft Flight Simulator\\Packages')]


In [4]:
import os
def find_aircrafts(package_path: str):
    aircraft_cfg_name = 'aircraft.cfg'
    flight_model_cfg_name = 'flight_model.cfg'

    aircrafts = []
    for path, _, files in os.walk(package_path):
        for name in files:
            if name.endswith(flight_model_cfg_name):
                aircrafts.append((path, os.path.join(path, aircraft_cfg_name), os.path.join(path, flight_model_cfg_name)))

    return aircrafts

aircrafts = find_aircrafts(packages[0][1])

In [12]:
import configparser

key_1 = 'GENERAL'
key_manufacturer = 'icao_manufacturer'
key_type = 'icao_type_designator'
key_model = 'icao_model'
keys_1 = [key_manufacturer, key_type, key_model]

key_2 = 'FLTSIM.0'
key_ceiling = 'ui_certified_ceiling'
key_max_range = 'ui_max_range'
key_autonomy = 'ui_autonomy'
keys_2 = [key_ceiling, key_max_range, key_autonomy]

def read_aircraft_cfg(file_name):
    aircraft_cfg = configparser.ConfigParser(strict=False, inline_comment_prefixes=(';'))
    aircraft_cfg.read(file_name)
    data = {}
    data.update({key:aircraft_cfg[key_1][key].replace('"', '') for key in keys_1})
    data.update({key:float(aircraft_cfg[key_2][key]) for key in keys_2})
    return data
    

In [13]:
KEY_3 = 'REFERENCE SPEEDS'
KEY_CRUISE_SPEED = 'cruise_speed'
KEYS_3 = [KEY_CRUISE_SPEED]
def read_flight_model_cfg(file_name):
    flight_model_cfg = configparser.ConfigParser(strict=False, inline_comment_prefixes=(';'), comment_prefixes=('#',';','/'))
    flight_model_cfg.read(file_name)
    data = {}
    data.update({key:float(flight_model_cfg[KEY_3][key]) for key in KEYS_3})
    return data


In [24]:
import openpyxl as op

wb = op.Workbook()
ws = wb.active
[ws.cell(row=1, column=j+1, value=v) for j,v in enumerate(['Manufacturer', 'Type', 'Model', 'Ceiling [ft]', 'Range [nm]', 'Duration [h]', 'Cruise speed [kt]'])]

row = 2
for aircraft in aircrafts:
    # print(aircraft[0])
    try:
        data_row = {}
        data_row.update(read_aircraft_cfg(aircraft[1]))
        data_row.update(read_flight_model_cfg(aircraft[2]))
        print(row, ','.join([f"{k}={v}" for k,v in data_row.items()]))
        [ws.cell(row=row, column=j+1, value=v) for j,v in enumerate(data_row.values())]
        row += 1
    except KeyError:
        # print('Key error', aircraft[0])
        pass
    pass

filter = ws.auto_filter
filter.ref = 'A:G'
fill = op.styles.PatternFill(start_color='FFCEF8FF', end_color='FFCEF8FF', fill_type='solid')
for c in range(7):
    ws.cell(row=1, column=c+1).fill = fill
ws.freeze_panes = "A2"

wb.save('aircrafts.xlsx')

2 icao_manufacturer=Piper,icao_type_designator=PA24,icao_model=Comanche,ui_certified_ceiling=20000.0,ui_max_range=1200.0,ui_autonomy=8.0,cruise_speed=160.0
3 icao_manufacturer=CESSNA,icao_type_designator=C152,icao_model=152,ui_certified_ceiling=14700.0,ui_max_range=415.0,ui_autonomy=5.0,cruise_speed=107.0
4 icao_manufacturer=CAMAIR,icao_type_designator=TNAV,icao_model=480 TWIN NAVION,ui_certified_ceiling=20000.0,ui_max_range=1113.0,ui_autonomy=8.0,cruise_speed=156.0
5 icao_manufacturer=Noorduyn,icao_type_designator=NORS,icao_model=Norseman,ui_certified_ceiling=17000.0,ui_max_range=800.0,ui_autonomy=5.0,cruise_speed=126.0
6 icao_manufacturer=Noorduyn,icao_type_designator=NORS,icao_model=Norseman,ui_certified_ceiling=17000.0,ui_max_range=800.0,ui_autonomy=5.0,cruise_speed=126.0
7 icao_manufacturer=Noorduyn,icao_type_designator=NORS,icao_model=Norseman,ui_certified_ceiling=17000.0,ui_max_range=800.0,ui_autonomy=5.0,cruise_speed=126.0
8 icao_manufacturer=HAWKER BEECHCRAFT,icao_type_designa

In [11]:
a = {}
a.update({'a':1})
a.update({'b':2})
print("|".join([f"{k}={v}" for k,v in a.items()]))
print(float('7.5'))
pass

a=1|b=2
7.5
